In [ ]:
# Install dependencies
!pip install torch torchvision opencv-python timm matplotlib

# Import library
import torch
import cv2
import numpy as np
from torchvision.transforms import Compose, ToTensor, Normalize
from google.colab.patches import cv2_imshow

# Load MiDaS model
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
midas.eval()

# Transformasi gambar
transform = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def process_depth(image):
    """Fungsi untuk menghitung depth dan memberi warna"""
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))  # Resize agar sesuai model

    img_tensor = transform(img).unsqueeze(0)

    # Prediksi depth
    with torch.no_grad():
        depth_map = midas(img_tensor)

    depth_map = depth_map.squeeze().numpy()
    depth_map = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_map = np.uint8(depth_map)

    # Tambahin colormap "jet" biar ada warna biru-ijo-merah
    depth_map_colored = cv2.applyColorMap(depth_map, cv2.COLORMAP_JET)

    return depth_map_colored


# Fungsi untuk menangkap gambar dari webcam
def capture_and_process():
    from IPython.display import display, Javascript
    from google.colab.output import eval_js
    from base64 import b64decode
    import PIL.Image
    import io

    # JavaScript untuk menangkap gambar dari webcam
    js = """
    async function takePhoto() {
      const video = document.createElement('video');
      const stream = await navigator.mediaDevices.getUserMedia({ video: true });

      document.body.appendChild(video);
      video.srcObject = stream;
      await new Promise((resolve) => video.onloadedmetadata = resolve);
      video.play();

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      document.body.removeChild(video);

      return canvas.toDataURL('image/jpeg', 0.8);
    }
    takePhoto();
    """

    display(Javascript(js))
    img_data = eval_js("takePhoto()")  # Ambil gambar dari webcam

    # Decode gambar dari base64
    img_bytes = b64decode(img_data.split(",")[1])
    image = PIL.Image.open(io.BytesIO(img_bytes))

    # Convert ke format OpenCV
    img_np = np.array(image)
    img_cv = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

    # Proses depth map
    depth_map = process_depth(img_cv)

    # Tampilkan hasilnya
    cv2_imshow(depth_map)
    print("Depth map berhasil dibuat!")

# Jalankan fungsi untuk capture dari webcam
capture_and_process()
